In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def lstsqk(x, y):
    return np.multiply(x, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0)

In [ ]:
def lstsqkerr(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt(np.multiply(y, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0) - lstsqk(x,y) ** 2)

# First

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Frequency').sort_values(by = ["m, кг"]).reset_index(drop = True)

# General Data

In [ ]:
l = 491e-3
d = 0.3
g0 = 9.81

In [ ]:
sigma_m_sys = 0.1
sigma_l_sys = 1e-3
sigma_d_sys = 0.01
sigma_f_sys = 1

In [ ]:
num = len(df1.index)

# Plot 

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

# Show grid
ax.grid(b = True, linestyle = '--')

# Set axis format to scientific
fig.canvas.draw()
plt.ticklabel_format(axis='both', style='sci', scilimits=(0,0))

plt.xlabel('$n$')
plt.ylabel('$\\nu_n$, Гц')

k_list = list()

# Interate over different tensions
for i in range(num):
    max_n = len(df1.iloc[i]['nu_1' : 'nu_9'])
    n_series = pd.Series([n + 1 for n in range(max_n)]).astype('float64')
    
    k_list.append(list([lstsqk(n_series, np.array(df1.iloc[i]['nu_1' : 'nu_9'])), np.sqrt(lstsqkerr(n_series, np.array(df1.iloc[i]['nu_1' : 'nu_9'])) ** 2 + sigma_f_sys ** 2)]))
    
    color = next(ax._get_lines.prop_cycler)['color']
    
    ax.scatter(n_series, df1.iloc[i]['nu_1' : 'nu_9'], color = color)
    
    x = np.linspace(0, max(n_series), 1000)
    
    T = df1.iloc[i]['m, кг'] * g0
    ax.plot(x, x * k_list[i][0], color = color, label = '$T = %s Н$' % (str(np.around(T, 2))))

# Set x tick spacing to 1
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))

# Set axis limits
ax.set_xlim(left = 0)
ax.set_ylim(bottom = 0)

# Set title
ax.set_title('Рис. 1. Зависимость $\\nu_n$ для различных натяжений $T$')

# Show plot
plt.legend()
plt.show()

fig.savefig('plots/figure_1.pdf')

In [ ]:
u_list = list()

for i in range(num):
    u = 2 * l * k_list[i][0]
    u_list.append([u, u * np.sqrt((sigma_l_sys / l) ** 2 + (k_list[i][1] / k_list[i][0]) ** 2)])

In [ ]:
u_list

In [ ]:
u_sq = [x[0] ** 2 for x in u_list]
t_list = [df1.iloc[i]['m, кг'] * g0 for i in range(num)]

In [ ]:
k_u_sq = lstsqk(np.array(t_list), np.array(u_sq))
k_u_sq_err = lstsqkerr(np.array(t_list), np.array(u_sq))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))

# Show grid
ax.grid(b = True, linestyle = '--')

# Set axis format to scientific
plt.ticklabel_format(axis='both', style='sci', scilimits=(0,0))
fig.canvas.draw()

# Plot fit line
x = np.linspace(0, max(t_list), 1000)
ax.plot(x, x * k_u_sq, label = '$u^2 = \\frac{1}{\\rho_1} T$')

# Dummy plots for legend
ax.plot([], [], ' ', label = '$\\frac{1}{\\rho_1} = %s$ $м/кг$ ' % str(np.round(k_u_sq, 2)))

# Plot data points
ax.scatter(t_list, u_sq)

# Set axis labels
plt.xlabel('$T, Н$')
plt.ylabel('$u^2, м/c$')

# Set axis limits
ax.set_xlim(left = 0)
ax.set_ylim(bottom = 0)

# Set title
ax.set_title('Рис. 2. Зависимость $u^2(T)$')

# Show plot
ax.legend()
plt.show()

fig.savefig('plots/figure_2.pdf')

In [ ]:
rho = k_u_sq ** -1
rho_sigma = rho * k_u_sq_err / k_u_sq

In [ ]:
rho

In [ ]:
rho_sigma